In [202]:

import torch
from torch.nn.functional import normalize
import pickle 
import torch.nn.functional as F 
import numpy as np 
from sklearn.metrics import f1_score
from torch import nn, optim
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import fetch_20newsgroups
import numpy as np
import pickle
import random
from scipy import sparse
import itertools
from scipy.io import savemat, loadmat
import re
import string
from gensim.models import Word2Vec
from sklearn.preprocessing import MultiLabelBinarizer
import torch
import pandas as pd
import sys
sys.path.insert(1, '/home/ec2-user/SageMaker/github/aspect_topic_modeling')
import swifter
from src.models.utils import remove_stopWords
from src.models.utils import get_wordnet_pos, remove_stopWords, get_emb, generate_emb, train, kld_normal, get_common_words, generate_bow
from collections import Counter
from models.NVDM import topic_covariance_penalty, sinkhorn_torch, NTM, negative_sampling_prior, optimal_transport_prior,  NormalParameter, get_mlp, EmbTopic, NSSTM, OTETM
from src.models.utils import get_wordnet_pos, remove_stopWords, get_emb, generate_emb, train, kld_normal, get_common_words
from hyperspherical_vae.distributions import VonMisesFisher
from hyperspherical_vae.distributions import HypersphericalUniform
from src.features.metric import diversity, get_topic_coherence

In [75]:
!pwd


/home/ec2-user/SageMaker/github/aspect_topic_modeling/notebooks


# 20News

In [9]:
# 20News Data
print('reading data...')
data = fetch_20newsgroups(subset = 'all')

reading data...


In [11]:
data.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [16]:
df = pd.DataFrame()
df['class'] = data['target']
df['text'] = data['data']

In [18]:
df.shape

(18846, 2)

In [21]:
df['text'] = df.swifter.apply(lambda x: ' '.join(remove_stopWords(x['text'])), axis=1)
df['clean_text']  = df.swifter.apply(lambda x: x['text'].split(), axis = 1)

common_words_ct = Counter([j for i in df['clean_text'].values for j in i])
common_words = get_common_words(common_words_ct, ct = 200)
word_track = {i: ind for ind, i in enumerate(common_words)}
index_track = {ind: i for ind, i in enumerate(common_words)}
df['index_num'] = df.apply(
            lambda x: [word_track[i] for i in x['clean_text'] if i in word_track], axis=1)


Dask Apply:   0%|          | 0/64 [00:00<?, ?it/s]

Dask Apply:   0%|          | 0/64 [00:00<?, ?it/s]

NameError: name 'Counter' is not defined

In [26]:
np.savetxt('/home/ec2-user/SageMaker/ORMCorpVoatp/ormcorpvoatp/ormcorpvoatp/data/Spherical-Text-Embedding/datasets/20news/text.txt',
           df['text'].values,
           fmt='%s')

In [29]:
#change it to any location you save your embeddings
vec = '/home/ec2-user/SageMaker/ORMCorpVoatp/ormcorpvoatp/ormcorpvoatp/data/Spherical-Text-Embedding/datasets/20news/jose.txt'
embed = generate_emb(vec, common_words).cpu()
X, indices = generate_bow(df = df, common_words = common_words)


array([10,  3, 17, ...,  3,  1,  7])

# TWN

In [34]:
from nltk.corpus import reuters
import nltk

In [35]:
nltk.download('reuters')

[nltk_data] Downloading package reuters to /home/ec2-user/nltk_data...


True

In [37]:
cats = reuters.categories()
total = len(reuters.paras())
total_multi = 0
for c in cats:
    lc = len(reuters.paras(categories=[c]))
    total_multi += lc
    print("%s ---- %d documents out of %d" % (c, lc, total))
print("Articles belong to %.4f categories on average" % ((total_multi * 1.0) / total))
print("There are %.4f articles per category on average" % ((total * 1.0) / len(cats)))


acq ---- 2661 documents out of 11887
alum ---- 62 documents out of 11887
barley ---- 58 documents out of 11887
bop ---- 120 documents out of 11887
carcass ---- 69 documents out of 11887
castor-oil ---- 2 documents out of 11887
cocoa ---- 81 documents out of 11887
coconut ---- 6 documents out of 11887
coconut-oil ---- 7 documents out of 11887
coffee ---- 141 documents out of 11887
copper ---- 73 documents out of 11887
copra-cake ---- 3 documents out of 11887
corn ---- 270 documents out of 11887
cotton ---- 65 documents out of 11887
cotton-oil ---- 3 documents out of 11887
cpi ---- 108 documents out of 11887
cpu ---- 6 documents out of 11887
crude ---- 658 documents out of 11887
dfl ---- 3 documents out of 11887
dlr ---- 223 documents out of 11887
dmk ---- 15 documents out of 11887
earn ---- 4211 documents out of 11887
fuel ---- 23 documents out of 11887
gas ---- 65 documents out of 11887
gnp ---- 146 documents out of 11887
gold ---- 131 documents out of 11887
grain ---- 640 documents ou

In [39]:
reuters.paras(categories=['dfl'])[0]

[['DUTCH',
  'OFFICIAL',
  'RATE',
  'CUT',
  'SEEN',
  'STILL',
  'LIKELY',
  'A',
  'cut',
  'of',
  'about',
  'half',
  'a',
  'percentage',
  'point',
  'in',
  'Dutch',
  'official',
  'interest',
  'rates',
  'is',
  'still',
  'in',
  'prospect',
  ',',
  'although',
  'economists',
  'said',
  'the',
  'timing',
  'would',
  'depend',
  'on',
  'Bundesbank',
  'moves',
  '.'],
 ['Speculation',
  'has',
  'been',
  'rife',
  'that',
  'the',
  'Dutch',
  'Central',
  'Bank',
  ',',
  'encouraged',
  'by',
  'a',
  'strong',
  'guilder',
  '/',
  'mark',
  'relationship',
  'and',
  'wide',
  'premiums',
  'for',
  'Dutch',
  'money',
  'and',
  'capital',
  'market',
  'rates',
  'over',
  'German',
  ',',
  'might',
  'lower',
  'rates',
  'without',
  'the',
  'Bundesbank',
  'moving',
  'first',
  '.'],
 ['Last',
  'month',
  ',',
  'the',
  'Central',
  'Bank',
  'lowered',
  'its',
  'special',
  'advances',
  'rate',
  'to',
  '5',
  '.',
  '1',
  'pct',
  'from',
  '5',


# Metrics

In [56]:
>>> from sklearn import metrics
>>> labels_true = [1, 1, 1, 0, 0, 0]
>>> labels_pred = [0, 2, 2, 2, 2, 2]

>>> metrics.mutual_info_score(labels_true, labels_pred)  



0.13230412471889824

In [47]:
labels_pred = [3, 3, 1, 1, 2, 2]
metrics.mutual_info_score(labels_true, labels_pred)  


0.4620981203732969

In [203]:
from collections import Counter
 
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][1]
   
List = [2, 1, 2, 2, 1, 3]
print(most_frequent(List))

3


In [271]:
from collections import defaultdict
def top_purity(labels_true, labels_pred):
    pred = np.unique(labels_pred)
    d = defaultdict(list)
    for i, j in zip(labels_pred, labels_true):
        d[i].append(j)
    ct = []
    for i in d:
        ct += [Counter(d[i]).most_common(1)[0][1]]
        #print(Counter(d[i]).most_common(1)[0], i, len(d[i]))
    print(ct)
    return np.sum(ct)/len(labels_pred)
        

In [270]:
top_purity(labels_true, labels_pred)

[952, 629, 18, 23, 9, 10, 1, 1, 1, 1, 1]


1646

# VNTM

In [410]:
class VNTM(nn.Module):
    """NTM that keeps track of output
    """
    def __init__(self, hidden, normal, h_to_z, topics, layer, top_number, penalty):
        super(VNTM, self).__init__()
        self.hidden = hidden
        #self.normal = normal
        self.h_to_z = h_to_z
        self.topics = topics
        self.output = None
        self.drop = nn.Dropout(p=0.5)
        self.fc_mean = nn.Linear(layer, top_number)
        self.fc_var = nn.Linear(layer, 1)
        self.num = top_number
        self.penalty = penalty
        #self.dirichlet = torch.distributions.dirichlet.Dirichlet((torch.ones(self.topics.k)/self.topics.k).cuda())
    def forward(self, x, n_sample=1, epoch = 0):
        h = self.hidden(x)
        h = self.drop(h)
        z_mean = self.fc_mean(h)
        z_mean = z_mean / z_mean.norm(dim=-1, keepdim=True)
        # the `+ 1` prevent collapsing behaviors
        z_var = F.softplus(self.fc_var(h)) + 1
        
        q_z = VonMisesFisher(z_mean, z_var)
        p_z = HypersphericalUniform(self.num - 1, device=device)
        kld = torch.distributions.kl.kl_divergence(q_z, p_z).mean()
        #print(q_z)
        #mu, log_sigma = self.normal(h)
        #identify how far it is away from normal distribution
        
        #print(kld.shape)
        rec_loss = 0
        for i in range(n_sample):
            #reparametrician trick
            z = q_z.rsample()
            #z = nn.Softmax()(z)
            #decode
            #print(z)
            
            z = self.h_to_z(10 * z)
            self.output = z
            #print(z)
            
            #get log probability for reconstruction loss
            log_prob = self.topics(z)
            rec_loss = rec_loss - (log_prob * x).sum(dim=-1)
        #average reconstruction loss
        rec_loss = rec_loss / n_sample
        #print(rec_loss.shape)
        minus_elbo = rec_loss + kld
        penalty, var, mean = topic_covariance_penalty(self.topics.topic_emb) 
        return {
            'loss': minus_elbo + penalty * self.penalty,
            'minus_elbo': minus_elbo,
            'rec_loss': rec_loss,
            'kld': kld
        }

    def get_topics(self):
        return self.topics.get_topics()


In [411]:
def train(model, X, batch_size, epoch, optimizer, scheduler):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    total_nll = 0.0
    total_kld = 0.0
    total_words = 0
    total_penalty = 0.0
    #size = epoch_size * batch_size
    indices = torch.randperm(X.shape[0])
    indices = torch.split(indices, batch_size)
    #print(indices)
    length = len(indices)
    for idx, ind in enumerate(indices):
        data_batch = torch.from_numpy(X[ind].toarray()).float().to(device)
        
        d = model(x = data_batch)
            
        
        
        total_nll += d['rec_loss'].sum().item() / batch_size
        total_kld += d['kld'].sum().item() / batch_size  
        #total_penalty += d['prior']  
#         if i < 3:
#             loss = d['minus_elbo']
#         else:
        loss = d['loss']

        optimizer.zero_grad()
        loss.sum().backward()
        optimizer.step()
        scheduler.step()
    print(total_nll/length, total_kld/length)
     


In [414]:
            numb_embeddings = 20
            hidden = get_mlp([X.shape[1], 128], nn.ReLU)
            normal = NormalParameter(128, 20)
            h_to_z = nn.Softmax()
            embedding = nn.Embedding(X.shape[1], 50)
            # p1d = (0, 0, 0, 10000 - company1.embeddings.shape[0]) # pad last dim by 1 on each side
            # out = F.pad(company1.embeddings, p1d, "constant", 0)  # effectively zero padding

            embedding.weight = torch.nn.Parameter(torch.Tensor(embed.float()))
            embedding.weight.requires_grad=False
            topics = EmbTopic(embedding = embedding,
                              k = numb_embeddings, normalize = False)



            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


            model = VNTM(hidden = hidden,
                        normal = normal,
                        h_to_z = h_to_z,
                        topics = topics,
                        layer = 128, 
                        top_number = numb_embeddings,
                        penalty = 0.5
                        ).to(device).float()
            # larger hidden size make topics more diverse
            #num_docs_train = 996318
            batch_size = 256
            optimizer = optim.Adam(model.parameters(), 
                                   lr=0.002, 
                                   weight_decay=1.2e-6)

            epochs = 20
            scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.05, steps_per_epoch=int(X.shape[0]/batch_size) + 1, epochs=epochs)

            
            for epoch in range(epochs):
                train(model, X,  batch_size, epoch, optimizer, scheduler)


/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/ipykernel/__main__.py:41: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


852.0981940192145 0.0023696595382864108
827.4855519887564 0.016863887210854807
798.30655690786 0.02468017753012277
776.2857109276024 0.028754686609514662
765.3556720630543 0.029878672338216693
759.0351537756018 0.03044270973249867
754.4659192884291 0.03450010261322196
752.3947927114126 0.036022817741173344
750.7459675556904 0.03327914277041281
749.6035197489971 0.033533523941563594
747.9836178341427 0.03254409529570792
747.6197880925359 0.031194197920125885
747.0079040527344 0.030624974267305555
746.4548442943676 0.029769035509309254


KeyboardInterrupt: 

In [415]:
#prior(softmax_top, indexes)
emb = model.topics.get_topics().cpu().detach().numpy()
topics =  [[index_track[ind] for ind in np.argsort(emb[i])[::-1][:25] ] for i in range(10)]
data_batch = torch.from_numpy(X.toarray()).float()
model.cpu()
z = model.hidden(data_batch)
z_mean = model.fc_mean(z)
z_mean = z_mean / z_mean.norm(dim=-1, keepdim=True)
z = model.h_to_z(z_mean)


labels_pred = torch.argmax(z, 1).numpy()
labels_true = df['class'].values
 

diversity_score = np.mean(diversity(topics))
#coherence_score = get_topic_coherence(X.toarray(), topics, word_track)
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20, random_state=0).fit(z.detach().numpy())
top_purity(labels_true, labels_pred), metrics.normalized_mutual_info_score(labels_true, labels_pred), top_purity(labels_true, kmeans.labels_), metrics.normalized_mutual_info_score(labels_true, kmeans.labels_)  
            

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/ipykernel/__main__.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[872, 792, 640, 339, 890, 204, 776, 788, 779, 734, 43, 71, 458, 130, 15]
[758, 643, 684, 186, 216, 851, 271, 689, 97, 630, 716, 202, 730, 319, 237, 213, 63, 108, 617, 160]


(0.3996073437334182,
 0.4839303169812979,
 0.44518730765149106,
 0.48417940107607277)

In [417]:
diversity(topics), np.mean(diversity(topics))

([0.4, 0.84, 0.32, 0.72, 0.64, 0.6, 0.4, 0.44, 0.92, 0.96], 0.624)

In [473]:
from sklearn.decomposition import LatentDirichletAllocation, NMF

lda = LatentDirichletAllocation(n_components=20, random_state=0)
lda.fit(X)
topic_index = np.argsort(lda.components_, axis=1)[:, -25:]
topics = [[index_track[j] for j in i] for i in topic_index]
np.mean(diversity(topics)), np.mean(get_topic_coherence(X.toarray(), topics, word_track))

(0.5040000000000001, -0.40466082922081026)

In [477]:
lda


LatentDirichletAllocation(n_components=20, random_state=0)

# NTM

In [232]:
def train(model, X, batch_size, epoch, optimizer):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    total_nll = 0.0
    total_kld = 0.0
    total_words = 0
    total_penalty = 0.0
    #size = epoch_size * batch_size
    indices = torch.randperm(X.shape[0])
    indices = torch.split(indices, batch_size)
    #print(indices)
    length = len(indices)
    for idx, ind in enumerate(indices):
        data_batch = torch.from_numpy(X[ind].toarray()).float().to(device)
        
        d = model(x = data_batch)
            
        
        
        total_nll += d['rec_loss'].sum().item() / batch_size
        total_kld += d['kld'].sum().item() / batch_size  
        #total_penalty += d['penalty'].sum().item() / batch_size  
#         if i < 3:
#             loss = d['minus_elbo']
#         else:
        loss = d['loss']

        optimizer.zero_grad()
        loss.sum().backward()
        optimizer.step()

    print(total_nll/length, total_kld/length)

In [249]:
topics = Topics(numb_embeddings, X.shape[1])
topics.get_topics()

tensor([[0.0005, 0.0005, 0.0004,  ..., 0.0003, 0.0004, 0.0005],
        [0.0005, 0.0004, 0.0004,  ..., 0.0004, 0.0005, 0.0004],
        [0.0003, 0.0003, 0.0005,  ..., 0.0005, 0.0005, 0.0005],
        ...,
        [0.0004, 0.0005, 0.0003,  ..., 0.0004, 0.0005, 0.0004],
        [0.0004, 0.0005, 0.0005,  ..., 0.0003, 0.0004, 0.0004],
        [0.0005, 0.0004, 0.0004,  ..., 0.0005, 0.0004, 0.0004]])

In [251]:
from models.NVDM import NTM
import gc
#labels = [[120], [1527], [1646], [2047], [727], [1624], [36], [32], [26], [92], [907], [652]]
numb_embeddings = 20
torch.cuda.empty_cache()
gc.collect()
hidden = get_mlp([X.shape[1], 64], nn.ReLU)
normal = NormalParameter(64, numb_embeddings)
h_to_z = nn.Softmax()
embedding = nn.Embedding(X.shape[1], 50)
# p1d = (0, 0, 0, 10000 - company1.embeddings.shape[0]) # pad last dim by 1 on each side
# out = F.pad(company1.embeddings, p1d, "constant", 0)  # effectively zero padding

# embedding.weight = torch.nn.Parameter(torch.Tensor(embed.float()))
# embedding.weight.requires_grad=False
topics = Topics(numb_embeddings, X.shape[1])



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = NTM(hidden = hidden,
            normal = normal,
            h_to_z = h_to_z,
            topics = topics
            ).to(device).float()
# larger hidden size make topics more diverse
#num_docs_train = 996318
batch_size = 256
optimizer = optim.Adam(model.parameters(), 
                       lr=0.002, 
                       weight_decay=1.2e-6)



epochs = 20
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.05, steps_per_epoch=int(X.shape[0]/batch_size) + 1, epochs=epochs)

for epoch in range(epochs):
    train(model, X,  batch_size, epoch, optimizer)


836.3474549989443 0.1607032038548307
819.3788678968275 0.03806465583526202
806.9361918681377 0.03929201230985691
797.9546310837204 0.048760768302993196
791.4055390229096 0.04600683428548478
786.7550077696104 0.038150564075221084
783.4681206780512 0.02321837155299412
781.0415558686127 0.025440513194110746
779.4308673755543 0.04452345746481237
778.4024130331503 0.0753821546048228
777.6905938225824 0.06864226998987834
777.3639212943413 0.023134106206959364
776.9615222827808 0.1023010356647491
776.8203562143686 0.06381778039881406
776.6001384838207 0.0499157341703697
776.7200680294552 0.10331941790236915
776.361110377956 0.13221442471793224
776.5740822456978 0.11337339315198462
776.1798528722815 0.162217427351877
776.2847937506598 0.10506580079704322


In [272]:
h = model.hidden(data_batch)
h = model.drop(h)
mu, log_sigma = model.normal(h)
z = model.h_to_z(mu)
labels_pred = torch.argmax(z, 1).numpy()
labels_true = df['class'].values
diversity_score = np.mean(diversity(topics))
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20, random_state=0).fit(z.detach().numpy())
top_purity(labels_true, labels_pred), metrics.mutual_info_score(labels_true, labels_pred), top_purity(labels_true, kmeans.labels_), metrics.mutual_info_score(labels_true, kmeans.labels_)  
            

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/ipykernel/__main__.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[964, 624, 19, 21, 9, 6, 1, 1, 1, 1]
[981, 67, 121, 14, 51, 58, 81, 120, 14, 8, 18, 4, 21, 12, 29, 7, 23, 4, 13, 5]


(0.08739255014326648,
 0.09698613336289995,
 0.08760479677385122,
 0.12068967614918263)

[0.64, 0.64, 0.72, 1.0, 0.84, 1.0, 0.88, 0.68, 0.76, 0.92]

array([list(['from', 'mamatha', 'devineni', 'ratnam', 'mr47', 'andrew', 'cmu', 'edu', 'subject', 'pen', 'fan', 'reaction', 'organization', 'post', 'office', 'carnegie', 'mellon', 'pittsburgh', 'pa', 'line', '12', 'nntp', 'post', 'host', 'po4', 'andrew', 'cmu', 'edu', 'i', 'be', 'sure', 'some', 'bashers', 'of', 'pen', 'fan', 'be', 'pretty', 'confuse', 'about', 'the', 'lack', 'of', 'any', 'kind', 'of', 'post', 'about', 'the', 'recent', 'pen', 'massacre', 'of', 'the', 'devil', 'actually', 'i', 'be', 'bit', 'puzzled', 'too', 'and', 'a', 'bit', 'relieved', 'however', 'i', 'be', 'go', 'to', 'put', 'an', 'end', 'to', 'non', 'pittsburghers', 'relief', 'with', 'a', 'bit', 'of', 'praise', 'for', 'the', 'pen', 'man', 'they', 'be', 'kill', 'those', 'devil', 'bad', 'than', 'i', 'think', 'jagr', 'just', 'show', 'you', 'why', 'he', 'be', 'much', 'good', 'than', 'his', 'regular', 'season', 'stats', 'he', 'be', 'also', 'a', 'lot', 'fo', 'fun', 'to', 'watch', 'in', 'the', 'playoff', 'bowman', 'should', 

# GSM

In [427]:
class GSM(NTM):
    def __init__(self, hidden, normal, h_to_z, topics, penalty):
        # h_to_z will output probabilities over topics
        super(GSM, self).__init__(hidden, normal, h_to_z, topics)
        self.penalty = penalty

    def forward(self, x, n_sample=1):
        stat = super(GSM, self).forward(x, n_sample)
        loss = stat['loss']
        penalty, var, mean = topic_covariance_penalty(self.topics.topic_emb)

        stat.update({
            'loss': loss + penalty * self.penalty,
            'penalty_mean': mean,
            'penalty_var': var,
            'penalty': penalty * self.penalty,
        })

        return stat

In [435]:
embed.shape

torch.Size([2394, 50])

In [459]:
from models.NVDM import NTM
import gc
#labels = [[120], [1527], [1646], [2047], [727], [1624], [36], [32], [26], [92], [907], [652]]
numb_embeddings = 20
torch.cuda.empty_cache()
gc.collect()
hidden = get_mlp([X.shape[1], 64], nn.ReLU)
normal = NormalParameter(64, numb_embeddings)
h_to_z = nn.Softmax()
embedding = nn.Embedding(X.shape[1], 50)
# p1d = (0, 0, 0, 10000 - company1.embeddings.shape[0]) # pad last dim by 1 on each side
# out = F.pad(company1.embeddings, p1d, "constant", 0)  # effectively zero padding

embedding.weight = torch.nn.Parameter(torch.ones(embed.shape))
# embedding.weight.requires_grad=False
#embedding.weight = torch.nn.Parameter()
embedding.weight.requires_grad=True
topics = EmbTopic(embedding = embedding,
                  
                  k = numb_embeddings, normalize = False)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = GSM(hidden = hidden,
            normal = normal,
            h_to_z = h_to_z,
            topics = topics,
            penalty = 1
        
            ).to(device).float()
# larger hidden size make topics more diverse
#num_docs_train = 996318
batch_size = 256
optimizer = optim.Adam(model.parameters(), 
                       lr=0.002, 
                       weight_decay=1.2e-6)



epochs = 20
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001, steps_per_epoch=int(X.shape[0]/batch_size) + 1, epochs=epochs)

for epoch in range(epochs):
    train(model, X,  batch_size, epoch, optimizer, scheduler)


/home/ec2-user/SageMaker/github/aspect_topic_modeling/models/NVDM.py:73: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  z = self.h_to_z(z)


ModuleAttributeError: 'Topics' object has no attribute 'topic_emb'

In [444]:
#prior(softmax_top, indexes)
emb = model.topics.get_topics().cpu().detach().numpy()
topics =  [[index_track[ind] for ind in np.argsort(emb[i])[::-1][:25] ] for i in range(20)]
data_batch = torch.from_numpy(X.toarray()).float()
model.cpu()
h = model.hidden(data_batch)
h = model.drop(h)
mu, log_sigma = model.normal(h)
z = model.h_to_z(mu)
labels_pred = torch.argmax(z, 1).numpy()
labels_true = df['class'].values
diversity_score = np.mean(diversity(topics))
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20, random_state=0).fit(z.detach().numpy())
top_purity(labels_true, labels_pred), metrics.normalized_mutual_info_score(labels_true, labels_pred), top_purity(labels_true, kmeans.labels_), metrics.normalized_mutual_info_score(labels_true, kmeans.labels_)  
            

/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/ipykernel/__main__.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[887, 863, 374, 417, 152, 954, 494, 126, 4, 12]
[252, 398, 151, 150, 573, 307, 234, 370, 249, 225, 195, 63, 315, 65, 254, 86, 100, 63, 246, 10]


(0.2272630796986098,
 0.29222802899938016,
 0.22848349782447203,
 0.2981286821704531)

In [445]:
np.mean(diversity(topics))


0.23600000000000004

In [428]:
from models.NVDM import NTM
import gc
#labels = [[120], [1527], [1646], [2047], [727], [1624], [36], [32], [26], [92], [907], [652]]
numb_embeddings = 20
torch.cuda.empty_cache()
gc.collect()
hidden = get_mlp([X.shape[1],  64], nn.ReLU)
normal = NormalParameter(64, numb_embeddings)
h_to_z = nn.Softmax()
embedding = nn.Embedding(X.shape[1], 50)
# p1d = (0, 0, 0, 10000 - company1.embeddings.shape[0]) # pad last dim by 1 on each side
# out = F.pad(company1.embeddings, p1d, "constant", 0)  # effectively zero padding

# embedding.weight = torch.nn.Parameter(torch.Tensor(embed.float()))
# embedding.weight.requires_grad=False
embedding.weight = torch.nn.Parameter(torch.Tensor(embed.float()))
embedding.weight.requires_grad=False
topics = EmbTopic(embedding = embedding,
                  k = numb_embeddings, normalize = False)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = GSM(hidden = hidden,
            normal = normal,
            h_to_z = h_to_z,
            topics = topics,
            penalty = 0
        
            ).to(device).float()
# larger hidden size make topics more diverse
#num_docs_train = 996318
batch_size = 256
optimizer = optim.Adam(model.parameters(), 
                       lr=0.002, 
                       weight_decay=1.2e-6)



epochs = 20
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001, steps_per_epoch=int(X.shape[0]/batch_size) + 1, epochs=epochs)

for epoch in range(epochs):
    train(model, X,  batch_size, epoch, optimizer, scheduler)




/home/ec2-user/SageMaker/github/aspect_topic_modeling/models/NVDM.py:73: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  z = self.h_to_z(z)


850.2488122888514 0.3557278318582354
848.9807928961676 0.36526529450674317
846.1792619035051 0.7726792844163405
842.8049926757812 1.4372556644517023
839.5003381677576 1.8342022106454179
835.8960084657411 2.2670185904245117
832.7783161885029 2.7266449058378064
829.2468360694679 3.1057512148006543
826.2967389080975 3.359474810394081
823.8363948512722 3.713141703927839
821.5484652132601 3.7246873443191117
819.6407594423036 3.865933759792431
817.949026159338 4.055987548183751
816.4779432142103 4.0030235052108765
815.4905651195629 4.030273927224649
814.7514087573902 4.010774493217468
814.2587771029085 4.141421417932253
814.0171698492926 4.032950752490276
813.7811163824957 4.0853450233871875
813.7942315178949 4.082697945672113


In [429]:
#prior(softmax_top, indexes)
emb = model.topics.get_topics().cpu().detach().numpy()
topics =  [[index_track[ind] for ind in np.argsort(emb[i])[::-1][:25] ] for i in range(20)]
data_batch = torch.from_numpy(X.toarray()).float()
model.cpu()
h = model.hidden(data_batch)
h = model.drop(h)
mu, log_sigma = model.normal(h)
z = model.h_to_z(mu)
labels_pred = torch.argmax(z, 1).numpy()
labels_true = df['class'].values
diversity_score = np.mean(diversity(topics))
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20, random_state=0).fit(z.detach().numpy())
top_purity(labels_true, labels_pred), metrics.normalized_mutual_info_score(labels_true, labels_pred), top_purity(labels_true, kmeans.labels_), metrics.normalized_mutual_info_score(labels_true, kmeans.labels_)  


/home/ec2-user/anaconda3/envs/pytorch_latest_p36/lib/python3.6/site-packages/ipykernel/__main__.py:9: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


[920, 116, 498, 710, 31, 47, 18, 100, 59, 26, 36, 36, 11, 7, 24, 4, 5, 2]
[158, 40, 166, 297, 235, 203, 287, 165, 307, 121, 10, 117, 194, 58, 51, 21, 102, 26, 23, 12]


(0.1406133927623899,
 0.13513739602163383,
 0.13758887827655736,
 0.10114286736005487)

In [431]:
np.mean(diversity(topics))

0.726

In [491]:
sys.path.insert(1, '/home/ec2-user/SageMaker/github/aspect_topic_modeling/pt-avitm')
from ptavitm.sklearn_api import ProdLDATransformer

In [545]:
model = ProdLDATransformer(
        cuda=device,
        batch_size=256,
        epochs=80,
        hidden1_dimension=64,
        hidden2_dimension=64,
        topics=20,
        lr=0.001,
        samples=50000)

In [546]:
model.fit(X)

In [547]:
result = model.transform(X)

In [548]:
        decoder_weight = model.autoencoder.decoder.linear.weight.detach().cpu()
        id2word = {index: str(index) for index in range(X.shape[1])}
        topics = [
            [item.item() for item in topic]
            for topic in decoder_weight.topk(min(model.score_num, X.shape[1]), dim=0)[
                1
            ].t()
        ]
 

In [549]:
diversity(topics)

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.8571428571428571,
 1.0,
 1.0,
 0.8571428571428571,
 1.0,
 0.7142857142857143,
 1.0,
 1.0,
 1.0,
 0.8571428571428571,
 1.0,
 0.5714285714285714]

In [556]:
labels_pred = result.argmax(1)
labels_true = df['class'].values
diversity_score = np.mean(diversity(topics))
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
kmeans = KMeans(n_clusters=20, random_state=0).fit(normalize(result, axis=1))
top_purity(labels_true, labels_pred), metrics.normalized_mutual_info_score(labels_true, labels_pred), top_purity(labels_true, kmeans.labels_), metrics.normalized_mutual_info_score(labels_true, kmeans.labels_)  


[103, 398, 243, 70, 225, 448, 265, 437, 53, 331, 336, 245, 384, 105, 182, 264, 100, 240, 344, 151]
[98, 510, 181, 339, 225, 577, 357, 313, 315, 596, 137, 452, 479, 266, 161, 241, 251, 191, 104, 173]


(0.2612756022498143,
 0.24761351519096145,
 0.3165658495171389,
 0.3046968220454173)

In [558]:
from gensim.matutils import Sparse2Corpus
corpus = Sparse2Corpus(X, documents_columns=False)


In [560]:
topics

[[1877, 1617, 2048, 1968, 1881, 1878, 1754],
 [835, 2095, 697, 1628, 2010, 711, 1833],
 [1370, 405, 1948, 1464, 2225, 1413, 2162],
 [2040, 2031, 1740, 215, 2033, 2032, 254],
 [1630, 910, 1076, 1935, 1938, 2168, 245],
 [208, 144, 1598, 1593, 1597, 1243, 1127],
 [2323, 1158, 2246, 830, 2002, 2014, 1210],
 [763, 764, 1336, 1567, 758, 462, 2159],
 [1112, 2272, 1346, 1100, 1239, 567, 1099],
 [2386, 2392, 2388, 2391, 2382, 2381, 2389],
 [1638, 1670, 1634, 1269, 2286, 166, 1671],
 [1461, 1460, 1427, 667, 1462, 1439, 1099],
 [557, 1614, 2157, 2377, 2345, 334, 1826],
 [436, 435, 438, 2175, 1034, 2258, 1986],
 [681, 1193, 2034, 599, 2239, 807, 1859],
 [1042, 1215, 1146, 1086, 1894, 1043, 1275],
 [772, 778, 1315, 774, 187, 797, 2223],
 [1742, 2088, 1744, 2039, 1562, 1745, 446],
 [1544, 1352, 417, 111, 1405, 335, 1068],
 [1562, 1542, 435, 91, 425, 438, 1541]]

In [561]:

corpus = Sparse2Corpus(X, documents_columns=False)
#decoder_weight = self.autoencoder.decoder.linear.weight.detach().cpu()
id2word = {index: str(index) for index in range(X.shape[1])}
topics = [
            [item.item() for item in topic]
            for topic in decoder_weight.topk(min(model.score_num, X.shape[1]), dim=0)[
                1
            ].t()
        ]
 
cm = CoherenceModel(
    topics=topics,
    corpus=corpus,
    dictionary=Dictionary.from_corpus(corpus, id2word),
    coherence="u_mass",
)



In [585]:
cm = CoherenceModel(
    topics=topics,
    texts = df.apply(lambda x:[str(i) for i in x['index_num']], axis = 1).values,
    corpus=corpus,
    dictionary=Dictionary.from_corpus(corpus, id2word),
    coherence='c_npmi',
)

cm.get_coherence()

-0.04799074373478669

In [586]:
cm.get_coherence_per_topic()

[0.010575433373172134,
 -0.08864524531280475,
 -0.29738757225699636,
 -0.12675700414382224,
 -0.00046802004933244707,
 -0.2907925052459905,
 -0.24912661770649866,
 0.09824602830557984,
 -0.07246353833620021,
 0.5782802533523147,
 -0.08915202093578214,
 0.19697654636731932,
 -0.209023759286811,
 -0.09121954098799176,
 -0.02777727205228651,
 0.039474022414436885,
 -0.0730726645682248,
 -0.036998336957972625,
 -0.09971504504233546,
 -0.13076801562550727]

TypeError: 'numpy.ndarray' object is not callable

In [485]:
class HiddenToLogNormal(nn.Module):
    def __init__(self, hidden_size, num_topics):
        super().__init__()
        self.fcmu = nn.Linear(hidden_size, num_topics)
        self.fclv = nn.Linear(hidden_size, num_topics)
        self.bnmu = nn.BatchNorm1d(num_topics)
        self.bnlv = nn.BatchNorm1d(num_topics)

    def forward(self, hidden):
        mu = self.bnmu(self.fcmu(hidden))
        lv = self.bnlv(self.fclv(hidden))
        dist = LogNormal(mu, (0.5 * lv).exp())
        return dist

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 7)

In [488]:
from torch.distributions import kl_divergence
class NTM(nn.Module):
    """NTM that keeps track of output
    """
    def __init__(self, hidden, normal, h_to_z, topics):
        super(NTM, self).__init__()
        self.hidden = hidden
        self.normal = normal
        self.h_to_z = h_to_z
        self.topics = topics
        self.output = None
        self.drop = nn.Dropout(p=0.5)
    def forward(self, x, n_sample=1):
        h = self.hidden(x)
        h = self.drop(h)
        posterior = self.normal(h)
        prior = standard_prior_like(posterior)
        #identify how far it is away from normal distribution
        kld = kl_divergence(posterior, prior).mean()
        #print(kld.shape)
        rec_loss = 0
        for i in range(n_sample):
            #reparametrician trick
            z = posterior.rsample().to(device)
            #decode
            
            z = z / z.sum(1, keepdim=True)
            self.output = z
            #print(z)
            #z = self.drop(z)
            #get log probability for reconstruction loss
            log_prob = self.topics(z)
            rec_loss = rec_loss - (log_prob * x).sum(dim=-1)
        #average reconstruction loss
        rec_loss = rec_loss / n_sample
        #print(rec_loss.shape)
        minus_elbo = rec_loss + kld

        return {
            'loss': minus_elbo,
            'minus_elbo': minus_elbo,
            'rec_loss': rec_loss,
            'kld': kld
        }

    def get_topics(self):
        return self.topics.get_topics()



In [489]:
import gc
from torch.distributions import LogNormal, Dirichlet
#labels = [[120], [1527], [1646], [2047], [727], [1624], [36], [32], [26], [92], [907], [652]]
numb_embeddings = 20
torch.cuda.empty_cache()
gc.collect()
hidden = get_mlp([X.shape[1],  64], nn.ReLU)
normal = HiddenToLogNormal(64, numb_embeddings)
h_to_z = nn.Softmax()
embedding = nn.Embedding(X.shape[1], 50)
# p1d = (0, 0, 0, 10000 - company1.embeddings.shape[0]) # pad last dim by 1 on each side
# out = F.pad(company1.embeddings, p1d, "constant", 0)  # effectively zero padding

# embedding.weight = torch.nn.Parameter(torch.Tensor(embed.float()))
# embedding.weight.requires_grad=False
embedding.weight = torch.nn.Parameter(torch.Tensor(embed.float()))
embedding.weight.requires_grad=False
topics = EmbTopic(embedding = embedding,
                  k = numb_embeddings, normalize = False)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = NTM(hidden = hidden,
            normal = normal,
            h_to_z = h_to_z,
            topics = topics
            
        
            ).to(device).float()
# larger hidden size make topics more diverse
#num_docs_train = 996318
batch_size = 256
optimizer = optim.Adam(model.parameters(), 
                       lr=0.002, 
                       weight_decay=1.2e-6)



epochs = 20
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.002, steps_per_epoch=int(X.shape[0]/batch_size) + 1, epochs=epochs)

for epoch in range(epochs):
    train(model, X,  batch_size, epoch, optimizer, scheduler)




nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan
nan nan


KeyboardInterrupt: 

In [471]:
#prior(softmax_top, indexes)
emb = model.topics.get_topics().cpu().detach().numpy()
topics =  [[index_track[ind] for ind in np.argsort(emb[i])[::-1][:25] ] for i in range(20)]
data_batch = torch.from_numpy(X.toarray()).float()
model.cpu()
h = model.hidden(data_batch)
h = model.drop(h)
mu = model.normal(h)
z = mu.mean
labels_pred = torch.argmax(z, 1).numpy()
labels_true = df['class'].values
diversity_score = np.mean(diversity(topics))
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=20, random_state=0).fit(z.detach().numpy())
top_purity(labels_true, labels_pred), metrics.normalized_mutual_info_score(labels_true, labels_pred), top_purity(labels_true, kmeans.labels_), metrics.normalized_mutual_info_score(labels_true, kmeans.labels_)  


[102, 102, 111, 617, 85, 52, 81, 56, 26, 27, 79, 28, 65, 76, 41, 84, 41, 53, 16, 24]
[176, 68, 63, 524, 352, 180, 109, 16, 28, 12, 3, 6, 4, 3, 4, 2, 6, 3, 5, 6]


(0.09370688740316248,
 0.031428944734180744,
 0.08330680250451024,
 0.03507299395058063)

In [472]:
import collections
collections.Counter(labels_pred)

Counter({10: 794,
         18: 924,
         7: 1669,
         17: 5989,
         14: 1185,
         9: 725,
         15: 1019,
         11: 470,
         6: 211,
         13: 292,
         4: 831,
         5: 371,
         0: 643,
         2: 914,
         19: 441,
         8: 1096,
         16: 523,
         12: 351,
         1: 195,
         3: 203})

# AG News

In [46]:
D = pd.read_csv('https://raw.githubusercontent.com/yumeng5/WeSTClass/master/agnews/dataset.csv', names = ['class', 'documents', 'text'])

In [51]:
D['clean_text'] = D['documents'] + D['text']

In [56]:
D['clean_text'] = D.swifter.apply(lambda x: ' '.join(remove_stopWords(x['documents'] + x['text'])).split(), axis=1)


Dask Apply:   0%|          | 0/64 [00:00<?, ?it/s]

In [57]:
#generate word 2 vec models
w2vmodel = Word2Vec(D['clean_text'].values,vector_size=200, window=10, negative = 5, min_count= 10)

#generate input for the model
vocab = list(set([j for i in init_docs for j in i if j in w2vmodel.wv]))
vocab = [''] + vocab
word_track = {i: ind for ind, i in enumerate(vocab)}
index_track = {ind: i for ind, i in enumerate(vocab)}
#pad the input


# RCV

In [58]:
from sklearn.datasets import fetch_rcv1
rcv1 = fetch_rcv1()

In [ ]:
#cannot use

In [ ]:
#backup dataset NYT and Arxiv

#https://github.com/yumeng5/JoSH/tree/master/datasets
